# Preparação da Base de Dados

Objetivo

#### Importação das bibliotecas necessárias

In [7]:
import pandas as pd
import random

#### Carregamento da base de dados em um dataframe do Pandas

In [5]:
df = pd.read_csv("bases/dados_tratados.csv")

## Criação de uma variável para o número de parcelas do empréstimo

Na base de dados original, não há uma coluna com a informação do número de parcelas do empréstimo. Para que a API de predição tenha mais fidelidade com a operação real, a coluna será criada e populada com base em critérios bem definidos.

A função abaixo calcula o número de parcelas para um registro da base de dados com base no valor do empréstimo, na finalidade e no score de crédito, aplicando um ruído aleatório para garantir a variablidade nos dados.

In [9]:
def calcular_numero_parcelas(registro):

    # O número base de parcelas é baseado em faixas de valores do empréstimo
    if registro['valor_emprestimo'] <= 7500:
        parcelas = 24
    elif registro['valor_emprestimo'] <= 15000:
        parcelas = 36
    elif registro['valor_emprestimo'] <= 25000:
        parcelas = 48
    else:
        parcelas = 60
    
    # O número de parcelas é ajustado de acordo com a finalidade
    finalidade = registro['finalidade_emprestimo']
    if finalidade == 'educacao' or finalidade == 'reforma_residencial':
        parcelas += 12
    elif finalidade == 'empreendimento' or finalidade == 'saude':
        parcelas += 6

    # É feito um ajuste fino com base no score de crédito
    if registro['score_credito'] > 650:
        parcelas -=6
    elif registro['score_credito'] < 550:
        parcelas += 6

    # Adicionando ruído aleatório
    ruido = random.randint(-4, 4)
    parcelas += ruido

    # Estabelecendo um intervalo para o número de parcelas
    if parcelas < 6:
        parcelas = 6
    elif parcelas > 84:
        parcelas = 84

    # Arredondando o número de parcelas para o múltiplo de 6 mais próximo
    parcelas_final = int(round(parcelas / 6.0) * 6)

    # O retorno garante o resultado mínimo após o arredondamento
    return max(6, parcelas_final)

In [10]:
df['numero_parcelas'] = df.apply(calcular_numero_parcelas, axis=1)

Verificando o resultado

In [11]:
df.head(10)

,idade,genero,escolaridade,renda_anual,experiencia_profissional_anos,tipo_moradia,valor_emprestimo,finalidade_emprestimo,taxa_juros_emprestimo,percentual_renda_comprometida,historico_credito_anos,score_credito,inadimplencia_anterior,status_emprestimo,numero_parcelas
0,22,feminino,mestrado,71948.0,0,aluguel,35000.0,pessoal,16.02,0.49,3,561,nao,aprovado,60
1,21,feminino,ensino_medio,12282.0,0,propria,1000.0,educacao,11.14,0.08,2,504,sim,negado,48
2,25,feminino,ensino_medio,12438.0,3,financiada,5500.0,saude,12.87,0.44,3,635,nao,aprovado,24
3,23,feminino,graduacao,79753.0,0,aluguel,35000.0,saude,15.23,0.44,2,675,nao,aprovado,60
4,24,masculino,mestrado,66135.0,1,aluguel,35000.0,saude,14.27,0.53,4,586,nao,aprovado,66
5,21,feminino,ensino_medio,12951.0,0,propria,2500.0,empreendimento,7.14,0.19,2,532,nao,aprovado,36
6,26,feminino,graduacao,93471.0,1,aluguel,35000.0,educacao,12.42,0.37,3,701,nao,aprovado,60
7,24,feminino,ensino_medio,95550.0,5,aluguel,35000.0,saude,11.11,0.37,4,585,nao,aprovado,60
8,24,feminino,tecnologo,100684.0,3,aluguel,35000.0,pessoal,8.90,0.35,2,544,nao,aprovado,72
9,21,feminino,ensino_medio,12739.0,0,propria,1600.0,empreendimento,14.74,0.13,3,640,nao,aprovado,36
